# Chapitre 0: Comprendre les Outils et l'Appel de Fonctions

Avant de plonger dans les systèmes agentiques avancés, nous devons comprendre un concept fondamental : **comment les LLMs utilisent les outils**.

## Qu'est-ce qu'un Outil ?

Les outils (aussi appelés "function calling") permettent aux LLMs de :
- Effectuer des calculs
- Interroger des bases de données
- Appeler des APIs externes
- Accéder à des informations en temps réel

**Concept clé** : Les LLMs n'exécutent pas directement les outils. Ils **génèrent des instructions structurées** qui nous indiquent quel outil appeler et avec quels paramètres. Nous exécutons ensuite l'outil et renvoyons le résultat au LLM.

## Configuration

In [ ]:
import json
from openai import AzureOpenAI
import os

# Configuration Azure OpenAI
config = {
    "openai_endpoint": "",
    "openai_key": "",
    "chat_deployment": ""
}

# Initialiser le client Azure OpenAI
client = AzureOpenAI(
    api_key=config["openai_key"],
    api_version="2024-02-01",
    azure_endpoint=config["openai_endpoint"]
)

print("✅ Client Azure OpenAI initialisé")

## Exemple d'Outil : Calculatrice Simple

Créons notre premier outil : une calculatrice simple.

In [ ]:
def calculatrice(operation: str, a: float, b: float) -> float:
    """Une calculatrice simple qui effectue des opérations de base."""
    if operation == "additionner":
        return a + b
    elif operation == "soustraire":
        return a - b
    elif operation == "multiplier":
        return a * b
    elif operation == "diviser":
        if b == 0:
            return "Erreur : Division par zéro"
        return a / b
    else:
        return f"Erreur : Opération inconnue '{operation}'"

# Testons-la
print(calculatrice("multiplier", 25, 4))

In [ ]:
prompt = "multiplie 9712439 et -138213" # Bonne réponse: -1342385331507
response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": prompt}]
)

print("\nRÉPONSE DU LLM (texte brut) :")
print("=" * 80)
reponse_texte = response.choices[0].message.content
print(reponse_texte)

**Questions :**
1. Pourquoi utiliser un outil de calculatrice ? Le réponse est-elle correcte? Essayer plusieurs fois.
2. Quelles idées avez-vous pour que le llm puisse utiliser la calculatrice?

### Approche manuelle: a. Prompter avec des outils

Construisons nous-mêmes le prompt qui décrit les outils disponibles :

In [ ]:
# Construisons manuellement le prompt qui décrit l'outil

description_outil = """**calculatrice** : Effectue des opérations arithmétiques de base
Paramètres :
- operation (string) : doit être "additionner", "soustraire", "multiplier", ou "diviser"
- a (number) : le premier nombre
- b (number) : le deuxième nombre

Pour utiliser un outil, tu DOIS répondre avec CE FORMAT JSON EXACT (rien d'autre) :
```json
{
  "outil": "calculatrice",
  "arguments": {
    "operation": "multiplier",
    "a": 25,
    "b": 4
  }
}
```"""

message_utilisateur = """Combien font 9712439 multiplié par -138213 ?"""

prompt = f"""Tu es un assistant qui a accès à des outils. Voici les outils disponibles :

{description_outil}

Si tu n'as pas besoin d'utiliser un outil, réponds normalement en texte.

Question de l'utilisateur : {message_utilisateur} """

print("=" * 80)
print("PROMPT COMPLET ENVOYÉ AU LLM :")
print("=" * 80)
print(prompt)
print("\n" + "=" * 80)

In [ ]:
# Appeler le LLM
response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": prompt}]
)

print("\nRÉPONSE DU LLM (texte brut) :")
print("=" * 80)
reponse_texte_1 = response.choices[0].message.content
print(reponse_texte_1)

**Questions :**
1. Le LLM a t-il compris la consigne? Essayer d'autres questions ci-dessous.

In [ ]:
message_utilisateur = """Essayer d'autres questions?"""

prompt = f"""Tu es un assistant qui a accès à des outils. Voici les outils disponibles :

{description_outil}

Si tu n'as pas besoin d'utiliser un outil, réponds normalement en texte.

Question de l'utilisateur : {message_utilisateur} """

In [ ]:
response = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": prompt}]
)

print("\nRÉPONSE DU LLM (texte brut) :")
print("=" * 80)
reponse_texte = response.choices[0].message.content
print(reponse_texte)

### Approche manuelle: b. Parser le json

In [ ]:
# Maintenant, on doit PARSER manuellement cette réponse
print("\n📝 PARSING MANUEL DE LA RÉPONSE :")
print("=" * 80)

# Le LLM a retourné du texte, on doit extraire le JSON
import re

try:
    # Chercher le JSON dans la réponse (souvent entre ```json et ```)
    json_match = re.search(r'```json\s*(\{.*?\})\s*```', reponse_texte_1, re.DOTALL)
    
    if json_match:
        json_str = json_match.group(1)
    else:
        # Peut-être que c'est du JSON direct
        json_str = reponse_texte.strip()
    
    # Parser le JSON
    appel_outil_parsed = json.loads(json_str)
    
    print("✅ JSON parsé avec succès !")
    print(json.dumps(appel_outil_parsed, indent=2, ensure_ascii=False))
    
    # Extraire les informations
    nom_outil = appel_outil_parsed.get("outil")
    arguments = appel_outil_parsed.get("arguments")
    
    print(f"\n🔧 Outil à appeler : {nom_outil}")
    print(f"📋 Arguments : {arguments}")
    
except Exception as e:
    print(f"❌ Erreur lors du parsing : {e}")
    print(f"⚠️  Problème : Le LLM n'a pas retourné le format attendu !")

**Questions :**
1. Le parser arrive-t-il a parser/interpreter le texte?
2. Pourquoi utiliser le format json?

### Approche manuelle: c. Executer l'outil avec les arguments

In [ ]:
# Bonne réponse: -1342385331507
# Exécuter l'outil
if nom_outil == "calculatrice":
    resultat = calculatrice(**arguments)
    print(f"✅ Résultat : {resultat}")

**Questions :**
1. La reponse est-elle correcte?
2. Est-ce que vous voyez comment ajouter d'autres outils?
3. Quelles sont les risques et

## Approche généraliste:
En pratiques les outils peuvent etre compliqués et en nombre assez important. La plupart des API de LLM integrent une notion d'outils afin de faciliter les interactions avec ceux-ci (intégration des descriptions au prompt, parsing des résultats)

### Description de l'Outil - Permet au LLM d'Utiliser Votre Outil

Le LLM a besoin de savoir :
1. Ce que fait l'outil
2. Quels paramètres il attend
3. Le type et format de chaque paramètre

Cela se fait via un **schéma d'outil** :

In [ ]:
outil_calculatrice = {
    "type": "function",
    "function": {
        "name": "calculatrice",
        "description": "Effectue des opérations arithmétiques de base (additionner, soustraire, multiplier, diviser) sur deux nombres.",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {
                    "type": "string",
                    "enum": ["additionner", "soustraire", "multiplier", "diviser"],
                    "description": "L'opération arithmétique à effectuer"
                },
                "a": {
                    "type": "number",
                    "description": "Le premier nombre"
                },
                "b": {
                    "type": "number",
                    "description": "Le deuxième nombre"
                }
            },
            "required": ["operation", "a", "b"]
        }
    }
}

print("Schéma de l'outil :")
print(json.dumps(outil_calculatrice, indent=2, ensure_ascii=False))

**Questions :**
1. Pourquoi utilise-t-on `enum` pour le paramètre operation ?
2. Que se passe-t-il si le LLM essaie de passer une chaîne pour le paramètre `a` ?
3. Pourquoi la description est-elle importante pour chaque paramètre ?

### Approche API

Maintenant faisons la même chose avec le paramètre `tools=` de l'API :

In [ ]:
# Appel avec l'API tools=
response_api = client.chat.completions.create(
    model=config["chat_deployment"],
    messages=[{"role": "user", "content": "Combien font 9712439 multiplié par -138213 ?"}],
    tools=[outil_calculatrice],  # ✅ On passe le schéma structuré
    tool_choice="auto" # On peut forcer ou désactiver l'utilisation de certains outils
)

message = response_api.choices[0].message

print("\nRÉPONSE DU LLM (structure) :")
print(f"- content : {message.content}")
print(f"- tool_calls : {message.tool_calls}")

if message.tool_calls:
    for tool_call in message.tool_calls:
        print(f"\n✅ L'API a AUTOMATIQUEMENT :")
        print(f"   - Parsé le nom de l'outil : {tool_call.function.name}")
        print(f"   - Parsé les arguments : {tool_call.function.arguments}")
        print(f"   - Validé que 'operation' est dans l'enum")
        print(f"   - Vérifié les types (a et b sont des numbers)")
        
        # Exécution
        arguments = json.loads(tool_call.function.arguments)
        resultat = calculatrice(**arguments)
        print(f"\n   🎯 Résultat : {resultat}")

### 💡 Réponse à la question : "Que fait `tools=` ?"

Le paramètre `tools=` :
1. **Prend votre schéma d'outil** (le JSON que vous définissez)
2. **L'intègre dans le contexte du LLM** (d'une manière optimisée par OpenAI/Azure)
4. **Garantit un format de sortie structuré** (tool_calls avec validation)
5. **Parse et valide automatiquement** les paramètres


### Approche API: Exécuter l'Outil et Retourner les Résultats

Le LLM ne fait que *demander* des appels d'outils. Nous devons :
1. Exécuter l'outil
2. Retourner le résultat au LLM
3. Laisser le LLM formuler une réponse finale

In [ ]:
def executer_appel_outil(nom_outil: str, arguments_outil: dict):
    """Exécute l'outil demandé."""
    if nom_outil == "calculatrice":
        return calculatrice(**arguments_outil)
    else:
        return f"Erreur : Outil inconnu '{nom_outil}'"

def boucle_complete_appel_outils(message_utilisateur: str, outils: list):
    """Boucle complète : LLM -> Exécution Outil -> LLM -> Réponse Finale."""
    print(f"Utilisateur : {message_utilisateur}\n")
    
    # Étape 1: Appel initial au LLM
    messages = [{"role": "user", "content": message_utilisateur}]
    response = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages,
        tools=outils,
        tool_choice="auto"
    )
    
    print("Décision du LLM :")
    
    message = response.choices[0].message
    
    # Vérifier si le LLM veut utiliser des outils
    if not message.tool_calls:
        print("  Pas d'outil nécessaire, réponse directe :")
        print(f"  {message.content}")
        return
    
    # Étape 2: Exécuter les outils
    messages.append(message)
    
    for tool_call in message.tool_calls:
        print(f"  Appel de l'outil : {tool_call.function.name}")
        arguments = json.loads(tool_call.function.arguments)
        print(f"  Avec arguments : {json.dumps(arguments, ensure_ascii=False)}")
        
        resultat = executer_appel_outil(tool_call.function.name, arguments)
        print(f"  Résultat : {resultat}\n")
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(resultat)
        })
    
    # Étape 3: Renvoyer les résultats au LLM pour une réponse finale
    reponse_finale = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages
    )
    
    print("Réponse Finale :")
    print(f"  {reponse_finale.choices[0].message.content}")

# Test de la boucle complète
boucle_complete_appel_outils("Combien font 1547 divisé par 23 ?", [outil_calculatrice])

### Comprendre le Flux

**Question** : Dans la cellule ci-dessus, identifiez les trois étapes :
1. Où le LLM décide-t-il d'utiliser un outil ?
2. Où exécutons-nous l'outil ?
3. Où le LLM voit-il le résultat et formule-t-il la réponse ?

Essayez avec différentes questions :

In [ ]:
# À VOTRE TOUR : Testez avec vos questions
cas_test = [
    "Combien font 892 multiplié par 47 ?",
    "Si j'ai 1000 euros et que je dépense 347, combien me reste-t-il ?",
    "Combien font 2 + 2 ?",  # Simple - utilisera-t-il quand même l'outil ?
]

for question in cas_test:
    print("\n" + "="*80)
    boucle_complete_appel_outils(question, [outil_calculatrice])

## Pièges Courants et Problèmes

### Piège 1: Descriptions d'Outils Mal Rédigées

In [ ]:
# MAUVAIS EXEMPLE : Description vague
mauvais_outil_calculatrice = {
    "type": "function",
    "function": {
        "name": "outil",
        "description": "outil",
        "parameters": {
            "type": "object",
            "properties": {
                "operation": {"type": "string"},
                "a": {"type": "number"},
                "b": {"type": "number"}
            },
            "required": ["operation", "a", "b"]
        }
    }
}

# Testez-le
print("Test avec une MAUVAISE description d'outil :")
response = appeler_llm_avec_outils("Combien font 25 fois 4 ?", [mauvais_outil_calculatrice])

message = response.choices[0].message
print(f"Reponse du LLM : {message}")
for tool_call in message.tool_calls:
    print()
    # Exécution
    arguments = json.loads(tool_call.function.arguments)
    resultat = calculatrice(**arguments)
    print(resultat)

**Question** : 
1. Qu'est ce qui ne va pas dans la description de l'outil ci dessus?
2. corrige le pour obtenir le résultat

### Piège 2: Paramètres Requis Manquants

In [ ]:
# Que se passe-t-il si l'utilisateur ne fournit pas assez d'informations ?
print("Test avec une question ambiguë :")
boucle_complete_appel_outils("Multiplie 5 par quelque chose", [outil_calculatrice])

# Le LLM va soit :
# 1. Demander des précisions
# 2. Faire des suppositions
# 3. Retourner une erreur

### Piège 3: Sur-utilisation des Outils

In [ ]:
# Test : Questions simples qui n'ont pas besoin d'outils
questions_simples = [
    "Combien font 2 + 2 ?",
    "Est-ce que 10 est plus grand que 5 ?",
    "Quel est le double de 50 ?"
]

print("Test de questions simples :")
for q in questions_simples:
    print(f"\nQuestion : {q}")
    response = appeler_llm_avec_outils(q, [outil_calculatrice])
    
    utilise_outil = response.choices[0].message.tool_calls is not None
    print(f"Utilise un outil : {utilise_outil}")

### Piège 4: Ne Pas Gérer les Erreurs d'Outils

In [ ]:
# Que se passe-t-il avec une division par zéro ?
print("Test de la gestion des erreurs :")
boucle_complete_appel_outils("Utilise la calculatrice pour voir combien font 100 divisé par 0 ?", [outil_calculatrice])

# Le LLM reçoit le message d'erreur et le gère avec élégance

## Exercice (facultatif) - Créez Votre Propre Outil

Maintenant c'est à vous ! Créez un outil simple et testez-le.

**Tâche** : Créez un outil "analyseur_texte" qui :
- Prend une chaîne de texte en entrée
- Retourne le nombre de mots et de caractères

Complétez le code ci-dessous :

In [ ]:
# Étape 1: Implémentez la fonction
def analyseur_texte(texte: str) -> dict:
    """Analyse un texte et retourne des statistiques."""
    # TODO: Implémentez cette fonction
    # Retournez un dictionnaire avec 'nombre_mots' et 'nombre_caracteres'
    pass

# Étape 2: Créez le schéma de l'outil
outil_analyseur_texte = {
    "type": "function",
    "function": {
        "name": "analyseur_texte",
        "description": "TODO: Écrivez une description claire",
        "parameters": {
            "type": "object",
            "properties": {
                # TODO: Définissez le paramètre 'texte'
            },
            "required": ["texte"]
        }
    }
}

# Étape 3: Testez-le
# TODO: Appelez le LLM avec une question comme "Combien de mots y a-t-il dans 'Bonjour le monde' ?"

### Solution (Exécutez après avoir tenté l'exercice)

In [ ]:
# Solution
def analyseur_texte(texte: str) -> dict:
    """Analyse un texte et retourne des statistiques."""
    mots = texte.split()
    return {
        "nombre_mots": len(mots),
        "nombre_caracteres": len(texte),
        "nombre_caracteres_sans_espaces": len(texte.replace(" ", ""))
    }

outil_analyseur_texte = {
    "type": "function",
    "function": {
        "name": "analyseur_texte",
        "description": "Analyse une chaîne de texte et retourne le nombre de mots, le nombre de caractères, et le nombre de caractères sans espaces.",
        "parameters": {
            "type": "object",
            "properties": {
                "texte": {
                    "type": "string",
                    "description": "Le texte à analyser"
                }
            },
            "required": ["texte"]
        }
    }
}

# Mise à jour de la fonction d'exécution d'outil
def executer_appel_outil_v2(nom_outil: str, arguments_outil: dict):
    if nom_outil == "calculatrice":
        return calculatrice(**arguments_outil)
    elif nom_outil == "analyseur_texte":
        return analyseur_texte(**arguments_outil)
    else:
        return f"Erreur : Outil inconnu '{nom_outil}'"

# Fonction de boucle mise à jour
def boucle_complete_appel_outils_v2(message_utilisateur: str, outils: list):
    print(f"Utilisateur : {message_utilisateur}\n")
    messages = [{"role": "user", "content": message_utilisateur}]
    
    response = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages,
        tools=outils,
        tool_choice="auto"
    )
    
    message = response.choices[0].message
    
    if not message.tool_calls:
        print(f"Réponse directe : {message.content}")
        return
    
    messages.append(message)
    
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        resultat = executer_appel_outil_v2(tool_call.function.name, arguments)
        print(f"Outil utilisé : {tool_call.function.name}")
        print(f"Résultat : {resultat}\n")
        
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": json.dumps(resultat, ensure_ascii=False) if isinstance(resultat, dict) else str(resultat)
        })
    
    reponse_finale = client.chat.completions.create(
        model=config["chat_deployment"],
        messages=messages
    )
    
    print("Réponse Finale :")
    print(f"  {reponse_finale.choices[0].message.content}")

# Testez-le
boucle_complete_appel_outils_v2(
    "Combien de mots y a-t-il dans la phrase : 'Le petit renard brun saute par-dessus le chien paresseux' ?",
    [outil_analyseur_texte]
)

## Partie 8: Outils Multiples et Sélection d'Outils

Que se passe-t-il quand le LLM a accès à plusieurs outils ? Testons :

In [ ]:
# Donner au LLM les deux outils
tous_les_outils = [outil_calculatrice, outil_analyseur_texte]

questions_test = [
    "Combien font 45 fois 67 ?",
    "Combien de mots y a-t-il dans 'Bonjour le monde' ?",
    "Compte les caractères dans 'Programmation Python' et multiplie par 2",  # Nécessite les deux !
]

for question in questions_test:
    print("\n" + "="*80)
    boucle_complete_appel_outils_v2(question, tous_les_outils)

**Observation Clé** : Le LLM peut :
1. Choisir le bon outil pour la tâche
2. Utiliser plusieurs outils en séquence
3. Combiner les résultats de différents outils

## Résumé des Bonnes Pratiques

### Écrire de Bonnes Descriptions d'Outils :
1. **Soyez spécifique** : Indiquez clairement ce que fait l'outil
2. **Utilisez des enums** : Restreignez les paramètres string aux valeurs valides
3. **Décrivez les paramètres** : Chaque paramètre a besoin d'une description claire
4. **Marquez les champs requis** : Spécifiez quels paramètres sont obligatoires

### Erreurs Courantes à Éviter :
1. Descriptions d'outils vagues
2. Descriptions de paramètres manquantes
3. Ne pas gérer les erreurs d'outils
4. Oublier de renvoyer les résultats des outils au LLM
5. Créer des outils pour des choses que le LLM peut faire directement

## Comprendre le Flux d'Appel d'Outils (Diagramme)

```
Question Utilisateur
     |
     v
LLM (avec définitions d'outils)
     |
     +---> Décision : Utiliser un outil ?
     |         |
     |         +---> Non : Retourner réponse directe
     |         |
     |         +---> Oui : Générer bloc tool_use
     v
Exécution de l'Outil (votre code)
     |
     v
Résultat de l'Outil
     |
     v
LLM (avec résultat de l'outil)
     |
     v
Réponse Finale à l'Utilisateur
```

## Conclusion

Vous comprenez maintenant :
- Comment les LLMs utilisent les outils via du JSON structuré
- Comment écrire des schémas d'outils
- La boucle complète d'appel d'outils
- Les pièges courants et comment les éviter

**Prochaines Étapes** : Dans le tutoriel principal, vous verrez comment LangGraph automatise cette boucle d'appel d'outils et permet des comportements agentiques complexes avec plusieurs outils travaillant ensemble !